In [1]:
import torch
from torch import nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# obtain mnist data and process
batch_size = 128
num_of_classes = 10

def get_indices(dataset,ind_array):
    indices =  []
    for i in range(len(dataset.targets)):
        for ind in ind_array:
            if dataset.targets[i] == ind:
                indices.append(i)
    return indices

dataset = MNIST('.', train=True, download=False,transform=transforms.ToTensor())

idx = get_indices(dataset, np.arange(num_of_classes))
data_loader_target = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))
data_loader_gan = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))

classes = np.arange(num_of_classes)

In [ ]:
# target model
from net_ import target, accuracy, train

net = target(num_of_classes)
criterion_tar = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

PATH = './target_models/basic_net_'+str(num_of_classes)+'classes.pth'
# load the model
net = target(num_of_classes)
net.load_state_dict(torch.load(PATH))

print('model accuracy: ', accuracy(net, data_loader_gan))

In [ ]:
from advGAN import advGAN
advgan = advGAN(net,gen_path_extra='VanillaAdvGAN')

In [ ]:
gen,disc = advgan.train(data_loader_gan)

In [ ]:

#             for img,adimg,out,adpred,p in zip(images,adv_images,outputs,adv_pred,pert):
#                 print('target model output: ', torch.argmax(out))
#                 print('target model output on adv example: ', torch.argmax(adpred))
#                 print('frobenius norm of pertubation: ',torch.norm(p,'fro'))
#                 fig, (ax1, ax2) = plt.subplots(1, 2)
#                 fig.suptitle('actual label: '+str(int(labels[0])))
#                 ax1.imshow(img.reshape(28,28).detach().numpy(),cmap='gray')
#                 ax2.imshow(adimg.reshape(28,28).detach().numpy(),cmap='gray')
#                 plt.show()
    